# Toxic Comment Classifier
---
For the full the report please refer to [Toxic Comment Classifier Report](insert link here).

## Housekeeping 

In the below section section we'll be importing python modules we'll be using throughout the project. As well as some jupyter notebook specific configuration. 

In [15]:
# Import libraries necessary for this project
import numpy as np
import pandas as pd
import matplot

# Pretty display for notebooks
%matplotlib inline

#load the Toxic Comment data 
data = pd.read_csv('./data/train.csv')

# print("Toxic comment coprus:")
# print("has {} rows with {} columns".format(*data.shape))
# print("{} are null / empty".format(data[data["comment"].is]))

## Data Analysis

Our first step is to understand the dataset we're working with. In this section we want to perfrom the following analysis of dataset: 
- Identify the distribution of labels
- Frequencies of various 